In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python3 "/content/drive/My Drive/DpProject/code/Model.py"

Accuracy (MNB) : 85.29%
Recall (MNB) : 0.8529411764705882%
Cross Validation Accuracy (MNB): 84.50%
Accuracy (RF) : 92.31%
Recall (RF) : 0.9230769230769231%
Cross Validation Accuracy (RF): 86.92%
Accuracy (LR) : 91.74%
Recall (LR) : 0.917420814479638%
Cross Validation Accuracy (LR): 89.19%
Accuracy (DT) : 91.97%
Recall (DT) : 0.919683257918552%
Cross Validation Accuracy (DT): 83.62%


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/DpProject')

In [ ]:
# importing all necessary libraries
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet
import requests
from googlesearch import search
import os
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from time import time
from collections import Counter
from statistics import mean
import operator
from xgboost import XGBClassifier
import math
from sklearn.linear_model import LogisticRegression

warnings.simplefilter("ignore")

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import requests
from bs4 import BeautifulSoup
from nltk.corpus import wordnet

def fetch_synonyms_thesaurus(term):
    synonyms = set()
    with requests.Session() as session:
        url = f'https://www.thesaurus.com/browse/{term}'
        try:
            response = session.get(url)
            response.raise_for_status()  # Raises HTTPError for bad requests
            soup = BeautifulSoup(response.content, "html.parser")
            container = soup.find('section', {'class': 'MainContentContainer'})
            if container:
                row = container.find('div', {'class': 'css-191l5o0-ClassicContentCard'})
                if row:
                    for x in row.find_all('li'):
                        synonyms.add(x.get_text())
        except requests.RequestException as e:
            print(f"Error fetching from Thesaurus.com: {e}")
    return synonyms

def fetch_synonyms_wordnet(term):
    synonyms = set()
    for syn in wordnet.synsets(term):
        synonyms.update(syn.lemma_names())
    return synonyms

def synonyms(term):
    """Fetches synonyms of a word from Thesaurus.com and WordNet."""
    thesaurus_synonyms = fetch_synonyms_thesaurus(term)
    wordnet_synonyms = fetch_synonyms_wordnet(term)
    return thesaurus_synonyms.union(wordnet_synonyms)


In [ ]:
def initialize_stop_words():
    try:
        return stopwords.words('english')
    except LookupError:
        nltk.download('stopwords')
        return stopwords.words('english')

def initialize_lemmatizer():
    return WordNetLemmatizer()

def initialize_tokenizer():
    return RegexpTokenizer(r'\w+')

stop_words = initialize_stop_words()
lemmatizer = initialize_lemmatizer()
splitter = initialize_tokenizer()



In [ ]:
def load_dataset(file_path):
    if os.path.exists(file_path):
        try:
            return pd.read_csv(file_path)
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
            return None
    else:
        print(f"File not found: {file_path}")
        return None

# Paths to the datasets
path_comb = "/content/drive/My Drive/DpProject/datasets/dis_sym_dataset_comb.csv"
path_norm = "/content/drive/My Drive/DpProject/datasets/dis_sym_dataset_norm.csv"

df_comb = load_dataset(path_comb)
df_norm = load_dataset(path_norm)

# Splitting features and labels for the combined dataset
if df_comb is not None:
    X = df_comb.iloc[:, 1:]
    Y = df_comb.iloc[:, 0:1]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Assuming X, Y are defined as before
# Flatten Y for model fitting
Y_flattened = Y.values.ravel()

# Initialize and fit the model
lr = LogisticRegression()
try:
    lr = lr.fit(X, Y_flattened)
    scores = cross_val_score(lr, X, Y_flattened, cv=5)
    print(f"Cross-validation scores: {scores}")
except Exception as e:
    print(f"Error in model fitting or cross-validation: {e}")

# Process for the second dataset
X = df_norm.iloc[:, 1:]
Y = df_norm.iloc[:, 0:1]
dataset_symptoms = list(X.columns)

# Function to preprocess user input symptoms
def preprocess_symptoms(user_symptoms):
    processed_symptoms = []
    for sym in user_symptoms:
        sym = sym.strip().replace('-', ' ').replace("'", '')
        sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
        processed_symptoms.append(sym)
    return processed_symptoms

# Take symptoms from user and preprocess them
user_input = input("Please list your symptoms, separating each one with a comma (,):\n")
processed_user_symptoms = preprocess_symptoms(user_input.lower().split(','))


Cross-validation scores: [0.74872666 0.88455008 0.92190153 0.94227504 0.96208263]
Please list your symptoms, separating each one with a comma (,):
fever,coughing,tire,headache


In [ ]:
def expand_symptoms(processed_symptoms):
    expanded_user_symptoms = []
    for user_sym in processed_symptoms:
        user_sym_words = user_sym.split()
        str_sym = set()

        # Generate combinations of words in the symptom
        for comb_len in range(1, len(user_sym_words) + 1):
            for subset in combinations(user_sym_words, comb_len):
                subset_str = ' '.join(subset)
                # Fetch synonyms for the subset
                subset_synonyms = synonyms(subset_str)
                str_sym.update(subset_synonyms)

        # Add the original symptom and create the expanded symptom string
        str_sym.add(user_sym)
        expanded_symptom = ' '.join(str_sym).replace('_', ' ')
        expanded_user_symptoms.append(expanded_symptom)

    return expanded_user_symptoms

user_symptoms = expand_symptoms(processed_user_symptoms)

print("Expanding query with related medical terms for symptoms entered:")
print(user_symptoms)


Expanding query with related medical terms for symptoms entered:
['fever febricity feverishness pyrexia febrility', 'cough coughing', 'weary sap tyre fag fag out wear down play out outwear bore jade tire out tire wear upon run down fatigue wear exhaust wear out pall', 'worry cephalalgia headache concern head ache vexation']


In [ ]:
user_symptom_sets = [set(sym.split()) for sym in user_symptoms]

# Loop over all the symptoms in the dataset
found_symptoms = set()
for data_sym in dataset_symptoms:
    data_sym_set = set(data_sym.split())

    # Check similarity with each user symptom
    for user_sym_set in user_symptom_sets:
        # Count the number of common elements
        common_elements_count = len(data_sym_set.intersection(user_sym_set))

        # Check if similarity score > 0.5 and add to found_symptoms if true
        if common_elements_count / len(data_sym_set) > 0.5:
            found_symptoms.add(data_sym)
            break

# Convert found symptoms from a set to a list
found_symptoms = list(found_symptoms)


In [ ]:
print("Here are the top matching symptoms based on search query:")
for idx, symp in enumerate(found_symptoms):
    print(f"{idx}: {symp}")

try:
    selected_indices = input("\nPlease select the symptoms that apply to you. Enter the numbers corresponding to these symptoms, separated by spaces:\n").split()
    selected_indices = [int(idx) for idx in selected_indices]
    final_symp = [found_symptoms[idx] for idx in selected_indices]
except (IndexError, ValueError):
    print("Invalid input. Please enter valid indices.")

dis_list = set(df_norm[df_norm[final_symp].sum(axis=1) > 0]['label_dis'])
counter_dict = {symptom: 0 for symptom in dataset_symptoms if symptom not in final_symp}

for dis in dis_list:
    for symptom in df_norm.loc[df_norm['label_dis'] == dis, dataset_symptoms].columns[df_norm.loc[df_norm['label_dis'] == dis, dataset_symptoms].any()]:
        if symptom in counter_dict:
            counter_dict[symptom] += 1

print("\nAdditional symptoms based on your selection:")
for symp, count in sorted(counter_dict.items(), key=lambda item: item[1], reverse=True):
    if count > 0:
        print(f"{symp} (Co-occurrences: {count})")

counter_list = []
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    if row:
        row[0].pop(0)
        for idx, val in enumerate(row[0]):
            if val != 0 and dataset_symptoms[idx] not in final_symp:
                counter_list.append(dataset_symptoms[idx])

if counter_list:
    dict_symp = dict(Counter(counter_list))
    dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1), reverse=True)

    print("Top 10 symptoms co-occurring with the selected ones, sorted by frequency:")
    for symp, count in dict_symp_tup[:10]:  # Slice to get only the top 10
        print(f"{symp}: {count}")
else:
    print("No additional co-occurring symptoms found.")



Here are the top matching symptoms based on search query:
0: fever
1: fatigue
2: coughing
3: headache

Please select the symptoms that apply to you. Enter the numbers corresponding to these symptoms, separated by spaces:
0 2 3

Additional symptoms based on your selection:
testicular pain (Co-occurrences: 17)
vomiting (Co-occurrences: 13)
barky cough (Co-occurrences: 9)
sore throat (Co-occurrences: 9)
confusion (Co-occurrences: 8)
maculopapular rash (Co-occurrences: 7)
diarrhea (Co-occurrences: 6)
feeling tired (Co-occurrences: 5)
nausea (Co-occurrences: 5)
shortness breath (Co-occurrences: 5)
swollen lymph node (Co-occurrences: 5)
chest pain (Co-occurrences: 4)
muscle weakness (Co-occurrences: 4)
runny nose (Co-occurrences: 4)
unintended weight loss (Co-occurrences: 4)
dizziness (Co-occurrences: 3)
large lymph node (Co-occurrences: 3)
red eye (Co-occurrences: 3)
seizure (Co-occurrences: 3)
tiredness (Co-occurrences: 3)
constipation (Co-occurrences: 2)
enlarged lymph node neck (Co-occur

In [ ]:
final_symp = []
found_symptoms = []
count = 0

for tup in dict_symp_tup:
    count += 1
    found_symptoms.append(tup[0])

    if count % 5 == 0 or count == len(dict_symp_tup):
        print("\nCommon co-occurring symptoms:")
        for idx, ele in enumerate(found_symptoms):
            print(f"{idx}: {ele}")

        user_input = input("Please review the symptoms: enter the indices of any you have (space-separated), type 'stop' to stop, or '-1' to skip:\n").lower().split()

        if user_input[0] == 'stop':
            break
        elif user_input[0] == '-1':
            found_symptoms = []
            continue

        # Validate and add selected symptoms
        try:
            selected_indices = [int(idx) for idx in user_input if idx.isdigit()]
            for idx in selected_indices:
                if 0 <= idx < len(found_symptoms):
                    final_symp.append(found_symptoms[idx])
        except ValueError:
            print("Invalid input. Please enter valid indices.")

        found_symptoms = []




Common co-occurring symptoms:
0: testicular pain
1: vomiting
2: barky cough
3: sore throat
4: confusion
Please review the symptoms: enter the indices of any you have (space-separated), type 'stop' to stop, or '-1' to skip:
3

Common co-occurring symptoms:
0: maculopapular rash
1: diarrhea
2: nausea
3: shortness breath
4: feeling tired
Please review the symptoms: enter the indices of any you have (space-separated), type 'stop' to stop, or '-1' to skip:
2 4

Common co-occurring symptoms:
0: swollen lymph node
1: chest pain
2: muscle weakness
3: runny nose
4: unintended weight loss
Please review the symptoms: enter the indices of any you have (space-separated), type 'stop' to stop, or '-1' to skip:
stop


In [ ]:
print("\nFinalized Symptoms for Prediction Analysis:")
sample_x = [0 for x in range(len(dataset_symptoms))]
for val in final_symp:
    print(val)
    sample_x[dataset_symptoms.index(val)] = 1


Finalized Symptoms for Prediction Analysis:
sore throat
nausea
feeling tired


In [ ]:
def diseaseDetail(term):
    query = term + ' wikipedia'
    ret = term + "\n"

    for sr in search(query, tld="co.in", num=1, stop=1, pause=2):
        if 'wikipedia' in sr:
            try:
                wiki = requests.get(sr)
                soup = BeautifulSoup(wiki.content, 'html5lib')
                info_table = soup.find("table", {"class": "infobox"})

                if info_table:
                    for row in info_table.find_all("tr"):
                        header = row.find("th", {"scope": "row"})
                        if header:
                            data = row.find("td")
                            if data:
                                # Remove citations (e.g., [1], [2], etc.)
                                data_text = re.sub(r'\[\d+\]', '', ' '.join(data.stripped_strings))
                                ret += f"{header.get_text()} - {data_text}\n"
                    return ret
            except requests.RequestException as e:
                return f"An error occurred: {e}"

    return f"No detailed information found for {term}."


In [ ]:
# Train the Logistic Regression model and make predictions
lr = LogisticRegression()
lr.fit(X, Y)
prediction = lr.predict_proba([sample_x])

# Get top k diseases
k = 5
diseases = list(set(Y['label_dis']))
diseases.sort()
topk = prediction[0].argsort()[-k:][::-1]

print(f"\nTop {k} diseases predicted based on symptoms")
topk_dict = {}

# Calculate custom probability for each disease
for idx, t in enumerate(topk):
    match_sym = set()
    row = df_norm.loc[df_norm['label_dis'] == diseases[t]].values.tolist()
    row[0].pop(0)

    for i, val in enumerate(row[0]):
        if val != 0:
            match_sym.add(dataset_symptoms[i])
    prob = (len(match_sym.intersection(set(final_symp))) + 1) / (len(set(final_symp)) + 1)
    prob *= mean(scores)
    topk_dict[t] = prob

# Display top k diseases
topk_index_mapping = {}
j = 0
topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
for key in topk_sorted:
    prob = topk_sorted[key] * 100
    print(f"{j} Disease name: {diseases[key]}, Probability: {round(prob, 2)}%")
    topk_index_mapping[j] = key
    j += 1

try:
    select = input("\nMore details about the disease? Enter index of disease or '-1' to discontinue and close the system:\n")
    if select != '-1':
        dis = diseases[topk_index_mapping[int(select)]]
        print()
        print(diseaseDetail(dis))
except ValueError:
    print("Invalid input. Please enter a valid index or '-1'.")



Top 5 diseases predicted based on symptoms
0 Disease name: Rubella, Probability: 66.89%
1 Disease name: Influenza, Probability: 66.89%
2 Disease name: Hepatitis E, Probability: 44.6%
3 Disease name: Hepatitis D, Probability: 44.6%
4 Disease name: Fibromyalgia, Probability: 44.6%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
1

Influenza
Other names - flu, the flu, grippe (French for flu)
Specialty - Infectious disease
Symptoms - Fever , runny nose , sore throat , muscle pain , headache , coughing , fatigue
Usual onset - 1–4 days after exposure
Duration - 2–8 days
Causes - Influenza viruses
Prevention - Hand washing , flu vaccines
Medication - Antiviral drugs such as oseltamivir
Frequency - 3–5 million severe cases per year  
Deaths - >290,000–650,000 deaths per year  

